# Setup for google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import sys
%cd "/content/drive/MyDrive/ColabNotebooks/Audio-ex3"
sys.path.append("/content/drive/MyDrive/ColabNotebooks/Audio-ex3")

In [ ]:
!git pull

In [ ]:
!pip install -r requirements.txt
%cd src

---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Logger

In [1]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

load_dotenv('./../.env')
API_TOKEN = os.environ.get("API_TOKEN")
PROJECT_NAME = os.environ.get("PROJECT_NAME")

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

---
# Loading Data

In [25]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm = loader.AudioDataModule()
dm.prepare_data()
dm.setup("fit")

# Create the classifier and train it

In [32]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)
config = classifier.default_config

trainer = classifier.train_func(config=config, dm=dm, logger=neptune_logger)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-92


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type    | Params | Mode 
-------------------------------------------
0 | lstm   | LSTM    | 16.9 M | train
1 | linear | Linear  | 55.3 K | train
2 | relu   | ReLU    | 0      | train
3 | loss   | CTCLoss | 0      | train
-------------------------------------------
17.0 M    Trainable params
0         Non-trainable params
17.0 M    Total params
67.822    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: input.size(-1) must be equal to input_size. Expected 13, got 32

# Validation

In [ ]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
importlib.reload(importlib.import_module("model_tuner"))
import model_tuner

model_tuner.tune_classifier(logger_config=logger_config, dm=dm)

2024-07-23 19:13:11,440	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


# Test the classifier

In [27]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

trainer.test(model, data_loader['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'NeuralNetwork' object has no attribute 'argmax_prob'